<a href="https://colab.research.google.com/github/Mini1521/Sentiment-Analysis/blob/main/Using_Microsoft_and_Tesla_Dataset_from_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stock Data- preprocessing

In [1]:
import pandas as pd

# Load the two CSV files
df1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Datasets/OG - Datasets/Company.csv")
df2 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Datasets/OG - Datasets/CompanyValues.csv")

# Merge the data on a common column (e.g., 'day_date')
merged_df = pd.merge(df1, df2, on="ticker_symbol", how="left")  # Use "outer", "left", or "right" if needed

# Save the merged dataset to a new CSV file
merged_df.to_csv("merged_data.csv", index=False)

print("Merged CSV saved as 'merged_data.csv'")


Merged CSV saved as 'merged_data.csv'


In [2]:
import pandas as pd

# Load the dataset (update the filename if needed)
df = pd.read_csv('/content/merged_data.csv')

df.rename(columns={'post_date': 'date'}, inplace=True)

# Convert 'day_date' to datetime format
df['day_date'] = pd.to_datetime(df['day_date'], errors='coerce')

# Filter data for the year 2020
df_2020 = df[df['day_date'].dt.year == 2019]
df_2020 = df_2020.sort_values(by='day_date', ascending=True)

# Separate data for Microsoft (MSFT) and Tesla (TSLA)
msft_data = df_2020[df_2020['ticker_symbol'] == 'MSFT']
tsla_data = df_2020[df_2020['ticker_symbol'] == 'TSLA']

# Drop rows with missing values
msft_data = msft_data.dropna()
tsla_data = tsla_data.dropna()


# print(" Stock Data Date Range:")
# print(f"Start Date: {msft_data['day_date'].min()} | End Date: {msft_data['day_date'].max()}")
# print(f"Start Date: {tsla_data['day_date'].min()} | End Date: {tsla_data['day_date'].max()}")

# Save each dataset separately
msft_data.to_csv("msft_2019.csv", index=False)
tsla_data.to_csv("tsla_2019.csv", index=False)

print("Data preprocessing complete.")
print("Saved: 'msft_2019.csv' and 'tsla_2019.csv'")


Data preprocessing complete.
Saved: 'msft_2019.csv' and 'tsla_2019.csv'


# Social Media - preprocessing

In [3]:
import numpy as np
import pandas as pd

# Load datasets
company_tweets = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Datasets/OG - Datasets/Company_Tweet.csv")
tweets = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Datasets/OG - Datasets/Tweet.csv")

In [4]:
# Merge tweets with company information
tweets = tweets.merge(company_tweets, how='left', on='tweet_id')

In [5]:
# Convert 'post_date' to datetime format
tweets['date'] = pd.to_datetime(tweets['post_date'], unit='s').dt.date
tweets['date'] = pd.to_datetime(tweets['date'], errors='coerce')
# tweets['time'] = pd.to_datetime(tweets['post_date'], unit='s').dt.time

In [6]:
tweets.to_csv("merged_tweets.csv", index=False)

In [7]:
tweets.drop(columns=['comment_num', 'retweet_num', 'like_num'], inplace=True)

In [8]:
# tweets.head()

In [9]:
# Filter for Microsoft (MSFT) and Tesla (TSLA)
tweets_filtered = tweets[tweets['ticker_symbol'].isin(['MSFT', 'TSLA'])]

# Keep only tweets from 2020
tweets_filtered = tweets_filtered[tweets_filtered['date'].dt.year == 2019]

In [10]:
# Split into separate datasets
msft_tweets = tweets_filtered[tweets_filtered['ticker_symbol'] == 'MSFT']
tsla_tweets = tweets_filtered[tweets_filtered['ticker_symbol'] == 'TSLA']

In [11]:
# Save to CSV files
msft_tweets.to_csv("msft_tweets_2019.csv", index=False)
tsla_tweets.to_csv("tsla_tweets_2019.csv", index=False)

# Cleaning of microsoft tweets

In [12]:
import re
import pandas as pd
import string
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [13]:
# Load Twitter dataset
tweets_df = pd.read_csv("/content/msft_tweets_2019.csv")

In [14]:
# rename columns
tweets_df.rename(columns={'body': 'Tweet'}, inplace=True)
tweets_df.rename(columns={'post_date': 'day_date'}, inplace=True)

In [15]:
# Check dataset info
# print(tweets_df.head())
#print(tweets_df.info())
# print(tweets_df.columns)

In [16]:
# Droping Duplicates
tweets_df.drop_duplicates(subset=['Tweet'], inplace=True)

In [17]:
# Handle Missing Values
tweets_df.dropna(subset=['Tweet'], inplace=True)  # Drop rows where 'Tweet' is empty

In [18]:
# Ensure 'date' is in datetime format
tweets_df['date'] = pd.to_datetime(tweets_df['date'], errors='coerce')

tweets_df = tweets_df.sort_values(by='date', ascending=True)

In [19]:
tweets_df.head()

,tweet_id,writer,day_date,Tweet,ticker_symbol,date
0,1079892580249337856,aaaamhim,1546301429,$NSPX 0041? ITS BREAKTHROUGH DRUG ON THE NEWS ...,MSFT,2019-01-01
36,1080132762974076928,trickmitch1,1546358693,NEVER wait for things to happen to you!Get off...,MSFT,2019-01-01
38,1080134005133996032,FinX_io,1546358989,$MSFT 2019-01-04 maxpain price 113.00,MSFT,2019-01-01
39,1080140795951251456,DZierler,1546360608,It’s a new year!Time for my 2019 #AR #VR #Voic...,MSFT,2019-01-01
40,1080146942125051904,ExactOptionPick,1546362073,251 Profit-Packed Option Trades Yearly--Delive...,MSFT,2019-01-01


In [20]:
# stop_words = set(stopwords.words('english'))
# def clean_text(text):
#     text = str(text).lower()  # Convert to lowercase
#     text = re.sub(r"http\S+|www\S+|https\S+", '', text)  # Removing URLs
#     text = re.sub(r'\@\w+|\#', '', text)  # Removing mentions and hashtags
#     text = re.sub(r'\d+', '', text)  # Removing numbers
#     text = re.sub(r'[^a-zA-Z0-9\s.,!?]', '', text)  # Keep some punctuation
#     tokens = word_tokenize(text)  # Tokenize text
#     tokens = [word for word in tokens if word not in stopwords.words('english')]  # Removing stopwords
#     return ' '.join(tokens)

# Initialize preprocessing tools
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = text.strip()
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]  # Lemmatization
    return ' '.join(words)


In [21]:
# Apply cleaning function
tweets_df['cleaned_tweet'] = tweets_df['Tweet'].astype(str).apply(clean_text)

In [22]:
#Saving the cleaned data for further analysis
tweets_df.to_csv("cleaned_msft_tweets.csv", index=False)

In [23]:
tweets_df.head()

,tweet_id,writer,day_date,Tweet,ticker_symbol,date,cleaned_tweet
0,1079892580249337856,aaaamhim,1546301429,$NSPX 0041? ITS BREAKTHROUGH DRUG ON THE NEWS ...,MSFT,2019-01-01,nspx breakthrough drug news httpsyoutubefigpni...
36,1080132762974076928,trickmitch1,1546358693,NEVER wait for things to happen to you!Get off...,MSFT,2019-01-01,never wait thing happen youget as go make happ...
38,1080134005133996032,FinX_io,1546358989,$MSFT 2019-01-04 maxpain price 113.00,MSFT,2019-01-01,msft maxpain price
39,1080140795951251456,DZierler,1546360608,It’s a new year!Time for my 2019 #AR #VR #Voic...,MSFT,2019-01-01,it’s new yeartime ar vr voicefirst prediction ...
40,1080146942125051904,ExactOptionPick,1546362073,251 Profit-Packed Option Trades Yearly--Delive...,MSFT,2019-01-01,profitpacked option trade yearlydelivered inbo...


# Cleaning of tesla tweets

In [24]:
# Load Twitter dataset
tweets_df = pd.read_csv("/content/tsla_tweets_2019.csv")

In [25]:
# # Check dataset info
# print(tweets_df.head())
# #print(tweets_df.info())
# print(tweets_df.columns)

In [26]:
# rename columns
tweets_df.rename(columns={'body': 'Tweet'}, inplace=True)
tweets_df.rename(columns={'post_date': 'day_date'}, inplace=True)

In [27]:
# Droping Duplicates
tweets_df.drop_duplicates(subset=['Tweet'], inplace=True)

In [28]:
# Handle Missing Values
tweets_df.dropna(subset=['Tweet'], inplace=True)  # Drop rows where 'Tweet' is empty

In [29]:
# Ensure 'date' is in datetime format
tweets_df['date'] = pd.to_datetime(tweets_df['date'], errors='coerce')

tweets_df = tweets_df.sort_values(by='date', ascending=True)

In [30]:
tweets_df.head()

,tweet_id,writer,day_date,Tweet,ticker_symbol,date
0,1079890068867817473,evdefender,1546300830,!! 8 Hours Left !!The picture you see here is...,TSLA,2019-01-01
302,1080181656881901568,fly4dat,1546370350,$TSLA reminder: there are 2 things #Tesla can'...,TSLA,2019-01-01
301,1080181493463429120,ValueAnalyst1,1546370311,I expect the poll result to prove wrong:$TSLA ...,TSLA,2019-01-01
300,1080181313058074624,fly4dat,1546370268,Most probable case: both.$TSLA $TSLAQ,TSLA,2019-01-01
299,1080181312307097600,AndysCycles,1546370268,$TSLA $TSRO $UPWK $USAC $W $WATT $WDAY $WHR $W...,TSLA,2019-01-01


In [31]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = text.strip()
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]  # Lemmatization
    return ' '.join(words)

In [32]:
# Apply cleaning function
tweets_df['cleaned_tweet'] = tweets_df['Tweet'].astype(str).apply(clean_text)

In [33]:
#Saving the cleaned data for further analysis
tweets_df.to_csv("cleaned_tsla_tweets.csv", index=False)

# Microsoft Articles - Preprocessing

In [34]:
import re
import pandas as pd
import string
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

# Load the dataset (Update the filename if needed)
article = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Datasets/OG - Datasets/msft_articles.csv")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [35]:
# Convert 'date' column to datetime format
article['date'] = pd.to_datetime(article['date'], errors='coerce')

# Filter for only the year 2020
year_2020 = article[article['date'].dt.year == 2020]

year_2020 = year_2020.sort_values(by='date', ascending=True)

# Drop rows with missing values
year_2020 = year_2020.dropna()

In [36]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = text.strip()
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]  # Lemmatization
    return ' '.join(words)

In [37]:
# Apply text cleaning to the 'text' column
year_2020['text'] = year_2020['text'].astype(str).apply(clean_text)

# Save the cleaned dataset
year_2020.to_csv("cleaned_msft_articles.csv", index=False)

In [38]:
year_2020.head()

,Unnamed: 0,date,text
1246,1246,2020-01-02,emerging detail suggest former nissan renault ...
1247,1247,2020-01-07,maker home speaker said google amazon stole te...
1238,1238,2020-01-07,company said would remove video altered artifi...
1229,1229,2020-01-08,stock rallied oil fell became clear iran unite...
1230,1230,2020-01-09,sp added nearly percent rally thanks combinati...


# Tesla Articles - Preprocessing

In [39]:
# Load the dataset (Update the filename if needed)
article = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets/OG - Datasets/tsla_articles.csv')

In [40]:
# Convert 'date' column to datetime format
article['date'] = pd.to_datetime(article['date'], errors='coerce')

# Filter for only the year 2020
year_2020 = article[article['date'].dt.year == 2020]

year_2020 = year_2020.sort_values(by='date', ascending=True)

# Drop rows with missing values
year_2020 = year_2020.dropna()

In [41]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = text.strip()
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]  # Lemmatization
    return ' '.join(words)

In [42]:
year_2020.head()

,Unnamed: 0,date,text
1241,1241,2020-01-01,"Sales doubled in 2019, but the industry’s tipp..."
1239,1239,2020-01-03,The electric-car maker ended a volatile year w...
1246,1246,2020-01-04,Americans are also continuing to buy ever bigg...
1244,1244,2020-01-09,The tech giant said it would not impose limits...
1251,1251,2020-01-10,Boeing released more than 100 pages of interna...


In [43]:
# Apply text cleaning to the 'text' column
year_2020['text'] = year_2020['text'].astype(str).apply(clean_text)

# Save the cleaned dataset
year_2020.to_csv("cleaned_tsla_articles.csv", index=False)

In [44]:
year_2020.head()

,Unnamed: 0,date,text
1241,1241,2020-01-01,sale doubled industry’s tipping point — severi...
1239,1239,2020-01-03,electriccar maker ended volatile year percent ...
1246,1246,2020-01-04,american also continuing buy ever bigger car p...
1244,1244,2020-01-09,tech giant said would impose limit advertising...
1251,1251,2020-01-10,boeing released page internal message max cong...


# Sentiment Analysis- microsoft

In [45]:
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from textblob import TextBlob

# Load datasets
tweets_df = pd.read_csv("/content/cleaned_msft_tweets.csv")
news_df = pd.read_csv("/content/cleaned_msft_articles.csv")

# Ensure text column is string type
tweets_df['Tweet'] = tweets_df['Tweet'].astype(str)
news_df['text'] = news_df['text'].astype(str)

In [46]:
# VADER Sentiment Analysis for Tweets

nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

# Function to get sentiment score from VADER
def get_vader_sentiment(text):
    score = sia.polarity_scores(text)['compound']
    return 1 if score > 0 else (-1 if score < 0 else 0)

# Apply VADER sentiment analysis to tweets
tweets_df['sentiment'] = tweets_df['Tweet'].apply(get_vader_sentiment)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [47]:
# TextBlob Sentiment Analysis for News

# Function to get sentiment score from TextBlob
def get_textblob_sentiment(text):
    score = TextBlob(text).sentiment.polarity
    return 1 if score > 0 else (-1 if score < 0 else 0)

# Apply TextBlob sentiment analysis to news articles
news_df['sentiment'] = news_df['text'].apply(get_textblob_sentiment)

# Save sentiment-labeled datasets
tweets_df.to_csv("sentiment_msft_tweets.csv", index=False)
news_df.to_csv("sentiment_msft_news.csv", index=False)

print("Sentiment analysis complete! Results saved.")

Sentiment analysis complete! Results saved.


In [48]:
# news_df.head()

In [49]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [50]:
# Load preprocessed datasets
tweets_df = pd.read_csv("/content/sentiment_msft_tweets.csv")
news_df = pd.read_csv("/content/sentiment_msft_news.csv")

In [51]:
# Train-test split
X_tweets, X_test_tweets, y_tweets, y_test_tweets = train_test_split(tweets_df['cleaned_tweet'], tweets_df['sentiment'], test_size=0.2)
X_news, X_test_news, y_news, y_test_news = train_test_split(news_df['text'], news_df['sentiment'], test_size=0.2)


In [52]:
# Create ML Pipelines for different models
def train_model(X_train, y_train, X_test, y_test, model):
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=5000)),
        ('classifier', model)
    ])
    pipeline.fit(X_train, y_train)
    predictions = pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f"Model {model.__class__.__name__} Accuracy: {accuracy:.4f}")
    return pipeline

In [53]:
# Train models
models = {
    'Naïve Bayes': MultinomialNB(),
    'SVM': SVC(kernel='linear'),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(n_estimators=100)
}

trained_models = {name: train_model(X_tweets, y_tweets, X_test_tweets, y_test_tweets, model) for name, model in models.items()}


Model MultinomialNB Accuracy: 0.7421
Model SVC Accuracy: 0.8872
Model DecisionTreeClassifier Accuracy: 0.8078
Model RandomForestClassifier Accuracy: 0.8458


SVM is the best model, followed by Random Forest Classifier.

Model MultinomialNB Accuracy: 0.7461
Model SVC Accuracy: 0.8903
Model DecisionTreeClassifier Accuracy: 0.8060
Model RandomForestClassifier Accuracy: 0.8510

In [54]:
# # Save the best-performing model for predictions
# import joblib
# best_model = trained_models['Random Forest']  # Assuming Random Forest performed best
# joblib.dump(best_model, "sentiment_model.pkl")

# print("Sentiment Analysis Complete")

# Sentiment Analysis- tesla

ALGORITHMS NOT WORKING !!!

In [54]:
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from textblob import TextBlob

# Load datasets
tweets_df = pd.read_csv("/content/cleaned_tsla_tweets.csv")
news_df = pd.read_csv("/content/cleaned_tsla_articles.csv")

# Ensure text column is string type
tweets_df['Tweet'] = tweets_df['Tweet'].astype(str)
news_df['text'] = news_df['text'].astype(str)

In [55]:
# VADER Sentiment Analysis for Tweets

nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

# # Function to get sentiment score from VADER
# def get_vader_sentiment(text):
#     score = sia.polarity_scores(text)['compound']
#     return 1 if score > 0 else (-1 if score < 0 else 0)

# # Apply VADER sentiment analysis to tweets
# tweets_df['sentiment'] = tweets_df['Tweet'].apply(get_vader_sentiment)

# Function to get sentiment score from VADER
def get_vader_sentiment(text):
    score = sia.polarity_scores(text)['compound']
    return 1 if score > 0 else (-1 if score < 0 else 0)

# Apply VADER sentiment analysis to tweets
tweets_df['sentiment'] = tweets_df['Tweet'].apply(get_vader_sentiment)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [56]:
# TextBlob Sentiment Analysis for News

# # Function to get sentiment score from TextBlob
# def get_textblob_sentiment(text):
#     score = TextBlob(text).sentiment.polarity
#     return 1 if score > 0 else (-1 if score < 0 else 0)

# # Apply TextBlob sentiment analysis to news articles
# news_df['sentiment'] = news_df['text'].apply(get_textblob_sentiment)

# news_df['sentiment'] = news_df['sentiment'].astype(int)

# Function to get sentiment score from TextBlob
def get_textblob_sentiment(text):
    score = TextBlob(text).sentiment.polarity
    return 1 if score > 0 else (-1 if score < 0 else 0)

# Apply TextBlob sentiment analysis to news articles
news_df['sentiment'] = news_df['text'].apply(get_textblob_sentiment)


# Save sentiment-labeled datasets
tweets_df.to_csv("sentiment_tsla_tweets.csv", index=False)
news_df.to_csv("sentiment_tsla_news.csv", index=False)

print("Sentiment analysis complete! Results saved.")

Sentiment analysis complete! Results saved.


In [57]:
print(news_df.head())

   Unnamed: 0        date                                               text  \
0        1241  2020-01-01  sale doubled industry’s tipping point — severi...   
1        1239  2020-01-03  electriccar maker ended volatile year percent ...   
2        1246  2020-01-04  american also continuing buy ever bigger car p...   
3        1244  2020-01-09  tech giant said would impose limit advertising...   
4        1251  2020-01-10  boeing released page internal message max cong...   

   sentiment  
0          0  
1          1  
2          1  
3          0  
4          1  


In [71]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Load preprocessed datasets
tweets_df = pd.read_csv("/content/sentiment_tsla_tweets.csv")
news_df = pd.read_csv("/content/sentiment_tsla_news.csv")

In [72]:
# Train-test split
X_tweets, X_test_tweets, y_tweets, y_test_tweets = train_test_split(tweets_df['cleaned_tweet'], tweets_df['sentiment'], test_size=0.2)
X_news, X_test_news, y_news, y_test_news = train_test_split(news_df['text'], news_df['sentiment'], test_size=0.2)

In [80]:
# Create ML Pipelines for different models
def train_model(X_train, y_train, X_test, y_test, model):
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=10000)),
        ('classifier', model)
    ])
    pipeline.fit(X_train, y_train)
    predictions = pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f"Model {model.__class__.__name__} Accuracy: {accuracy:.4f}")
    return pipeline

In [81]:
tweets_df.dropna(subset=['cleaned_tweet'], inplace=True)
news_df.dropna(subset=['text'], inplace=True)

# Ensure text columns are properly formatted
tweets_df['cleaned_tweet'] = tweets_df['cleaned_tweet'].astype(str)
news_df['text'] = news_df['text'].astype(str)

# Remove empty strings
tweets_df = tweets_df[tweets_df['cleaned_tweet'].str.strip() != ""]
news_df = news_df[news_df['text'].str.strip() != ""]

# Verify again
print("Final Check: Missing in tweets:", tweets_df['cleaned_tweet'].isna().sum())
print("Final Check: Missing in news:", news_df['text'].isna().sum())

Final Check: Missing in tweets: 0
Final Check: Missing in news: 0


In [94]:
# # Train models
# models = {
#     'Naïve Bayes': MultinomialNB(),
#     'SVM': SVC(kernel='linear'),
#     'Decision Tree': DecisionTreeClassifier(),
#     'Random Forest': RandomForestClassifier(n_estimators=100)
# }

# # Now train your ML model
# train_models = {name: train_model(A_tweets, b_tweets, A_test_tweets, b_test_tweets, model) for name, model in models.items()}

# Train models
# models = {
#     'Naïve Bayes': MultinomialNB(),
#     'SVM': SVC(kernel='linear'),
#     'Decision Tree': DecisionTreeClassifier(),
#     'Random Forest': RandomForestClassifier(n_estimators=5000)
# }

# trained_models = {name: train_model(X_tweets, y_tweets, X_test_tweets, y_test_tweets, model) for name, model in models.items()}


# Merging - Microsoft


In [ ]:
import pandas as pd

# Load datasets
stock_data = pd.read_csv("/content/msft_2019.csv")
tweets_sentiment = pd.read_csv("/content/sentiment_msft_tweets.csv")
news_sentiment = pd.read_csv("/content/sentiment_msft_news.csv")

In [ ]:
# Convert date columns to datetime
stock_data['day_date'] = pd.to_datetime(stock_data['day_date'])
tweets_sentiment['date'] = pd.to_datetime(tweets_sentiment['date'])
news_sentiment['date'] = pd.to_datetime(news_sentiment['date'])

In [ ]:
# Extract Month-Day format (MM-DD)
stock_data['month_day'] = stock_data['day_date'].dt.strftime('%m-%d')
tweets_sentiment['month_day'] = tweets_sentiment['date'].dt.strftime('%m-%d')
news_sentiment['month_day'] = news_sentiment['date'].dt.strftime('%m-%d')

In [ ]:
# # Drop original 'date' columns (Optional)
# stock_data.drop(columns=['day_date'], inplace=True)
# tweets_sentiment.drop(columns=['date'], inplace=True)
# news_sentiment.drop(columns=['date'], inplace=True)

In [ ]:
# Stock & Tweets
stock_tweet_merged = stock_data.merge(tweets_sentiment, on='month_day', how='left')
stock_tweet_merged.fillna(0, inplace=True)
stock_tweet_merged = stock_tweet_merged.rename(columns={'sentiment_vader': 'tweet_sentiment'})
stock_tweet_merged.to_csv("Merge_MSFT_TWEET.csv", index=False)

In [ ]:
# Stock & News
stock_news_merged = stock_data.merge(news_sentiment, on='month_day', how='left')
stock_news_merged.fillna(0, inplace=True)
stock_news_merged = stock_news_merged.rename(columns={'sentiment': 'news_sentiment'})
stock_news_merged.to_csv("Merge_MSFT_NEWS.csv", index=False)

print(" Merging done! Two separate files created: one for Tweets, one for News.")

# Merging - Tesla

In [ ]:
import pandas as pd

# Load datasets (assuming they are already preprocessed)
stock_data = pd.read_csv("/content/tsla_2019.csv")
tweets_sentiment = pd.read_csv("/content/sentiment_tsla_tweets.csv")
news_sentiment = pd.read_csv("/content/sentiment_tsla_news.csv")

In [ ]:
# Convert date columns to datetime
stock_data['day_date'] = pd.to_datetime(stock_data['day_date'])
tweets_sentiment['date'] = pd.to_datetime(tweets_sentiment['date'])
news_sentiment['date'] = pd.to_datetime(news_sentiment['date'])

In [ ]:
# Extract Month-Day format (MM-DD)
stock_data['month_day'] = stock_data['day_date'].dt.strftime('%m-%d')
tweets_sentiment['month_day'] = tweets_sentiment['date'].dt.strftime('%m-%d')
news_sentiment['month_day'] = news_sentiment['date'].dt.strftime('%m-%d')

In [ ]:
# Stock & Tweets
stock_tweet_merged = stock_data.merge(tweets_sentiment, on='month_day', how='left')
stock_tweet_merged.fillna(0, inplace=True)
stock_tweet_merged = stock_tweet_merged.rename(columns={'sentiment_vader': 'tweet_sentiment'})
stock_tweet_merged.to_csv("Merge_TSLA_TWEET.csv", index=False)

In [ ]:
# Stock & News
stock_news_merged = stock_data.merge(news_sentiment, on='month_day', how='left')
stock_news_merged.fillna(0, inplace=True)
stock_news_merged = stock_news_merged.rename(columns={'sentiment': 'news_sentiment'})
stock_news_merged.to_csv("Merge_TSLA_NEWS.csv", index=False)

print(" Merging done! Two separate files created: one for Tweets, one for News.")

# Data Analysis

In [ ]:
import pandas as pd
import numpy as np

# Load both datasets
stock_tweet = pd.read_csv("/content/Merge_MSFT_TWEET.csv")
stock_news = pd.read_csv("/content/Merge_MSFT_NEWS.csv")

# Convert 'month_day' to datetime format
stock_tweet['month_day'] = pd.to_datetime(stock_tweet['month_day'], format='%m-%d')
stock_news['month_day'] = pd.to_datetime(stock_news['month_day'], format='%m-%d')

# Sort data chronologically
stock_tweet = stock_tweet.sort_values('month_day')
stock_news = stock_news.sort_values('month_day')

# Feature Engineering
for df in [stock_tweet, stock_news]:
    df['5_day_avg'] = df['close_value'].rolling(window=5).mean()
    df['10_day_avg'] = df['close_value'].rolling(window=10).mean()
    df['volatility'] = df['close_value'].pct_change().rolling(window=5).std()
    df.fillna(0, inplace=True)  # Fill missing values

# Save new files
stock_tweet.to_csv("MSFT_FEATURED_TWEET.csv", index=False)
stock_news.to_csv("MSFT_FEATURED_NEWS.csv", index=False)

print(" Feature Engineering Completed for Both Files!")


In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Load Tweet Data
tweet_data = pd.read_csv("/content/MSFT_FEATURED_TWEET.csv")

# Scale Features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(tweet_data[['close_value', 'sentiment', '5_day_avg', '10_day_avg', 'volatility']])

# Prepare Data for LSTM
X, y = [], []
for i in range(30, len(scaled_data)):
    X.append(scaled_data[i-30:i])
    y.append(scaled_data[i, 0])

X, y = np.array(X), np.array(y)

# Build LSTM Model
model_tweet = Sequential()
model_tweet.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model_tweet.add(Dropout(0.2))
model_tweet.add(LSTM(units=50))
model_tweet.add(Dropout(0.2))
model_tweet.add(Dense(units=1))

model_tweet.compile(optimizer='adam', loss='mean_squared_error')
model_tweet.fit(X, y, epochs=10, batch_size=32)

# Save Model
model_tweet.save("LSTM_MSFT_TWEET.h5")

print(" LSTM Model Trained for Tweet Sentiment!")


In [ ]:
# Load News Data
news_data = pd.read_csv("/content/MSFT_FEATURED_NEWS.csv")

# Scale Features
scaled_news_data = scaler.fit_transform(news_data[['close_value', 'news_sentiment', '5_day_avg', '10_day_avg', 'volatility']])

# Prepare Data for LSTM
X_news, y_news = [], []
for i in range(30, len(scaled_news_data)):
    X_news.append(scaled_news_data[i-30:i])
    y_news.append(scaled_news_data[i, 0])

X_news, y_news = np.array(X_news), np.array(y_news)

# Build LSTM Model
model_news = Sequential()
model_news.add(LSTM(units=50, return_sequences=True, input_shape=(X_news.shape[1], X_news.shape[2])))
model_news.add(Dropout(0.2))
model_news.add(LSTM(units=50))
model_news.add(Dropout(0.2))
model_news.add(Dense(units=1))

model_news.compile(optimizer='adam', loss='mean_squared_error')
model_news.fit(X_news, y_news, epochs=20, batch_size=32)

# Save Model
model_news.save("LSTM_MSFT_NEWS.h5")

print("✅ LSTM Model Trained for News Sentiment!")


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Predictions
tweet_pred = model_tweet.predict(X)
news_pred = model_news.predict(X_news)

# Evaluation
def evaluate_model(y_true, y_pred, model_name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    print(f"{model_name} - MAE: {mae:.4f}, RMSE: {rmse:.4f}")

evaluate_model(y, tweet_pred, "LSTM Tweet Model")
evaluate_model(y_news, news_pred, "LSTM News Model")
# y_pred = model_tweet.predict(X_test)  # Ensure model exists and has been trained